<a href="https://colab.research.google.com/github/JeonJongDae/dinner_stock_AI/blob/main/3%EC%A3%BC%EC%B0%A8/%EC%98%AC%EC%8B%9C%EC%A6%8C_%ED%8F%AC%ED%8A%B8%ED%8F%B4%EB%A6%AC%EC%98%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Matplotlib 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')      

In [1]:
from pandas_datareader import data
import yfinance as yfin
yfin.pdr_override()

In [3]:
def create_portfolio_base():
    category = ['채권', '금', '원자재', '부동산', '국내주식', '해외주식']
    ratio = [10, 20, 20, 10, 20, 20]
    ticker = ['TTT', # 미국 국채 레버러지
              'GLD', # 금
              'BDC', # 원자재
              'VNQ', #부동산
              'EEM', #개발도상국 주식
              'SPY' #미국 주식
           
            ]

    df = pd.DataFrame({'자산군': category,
                    '세팅비중': ratio,
                    '티커': ticker})
    return df

base_df = create_portfolio_base()
base_df

,자산군,세팅비중,티커
0,채권,10,TTT
1,금,20,GLD
2,원자재,20,BDC
3,부동산,10,VNQ
4,국내주식,20,EEM
5,해외주식,20,SPY


In [4]:
fig = px.pie(base_df, values = '세팅비중', names='자산군',
             title='맞춤형 포트폴리오', template="plotly_white")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [5]:
def set_monthly_pred_investment(df, total):
    invest_ratio = []
    for i in df['세팅비중']:
        invest_ratio.append(total*i/100)
    df['예상투자금'] = invest_ratio
    return df

In [6]:
df = set_monthly_pred_investment(base_df, 100000000)
df

,자산군,세팅비중,티커,예상투자금
0,채권,10,TTT,10000000.0
1,금,20,GLD,20000000.0
2,원자재,20,BDC,20000000.0
3,부동산,10,VNQ,10000000.0
4,국내주식,20,EEM,20000000.0
5,해외주식,20,SPY,20000000.0


In [15]:
def set_etf_name(df):
    name = []
    for i in df['티커']:
        ticker = yfin.Ticker(i)
        name.append(ticker.info["longName"])
        print(name)
    df['종목명'] = name
    return df

In [16]:
df = set_etf_name(df)
df

['ProShares UltraPro Short 20+ Year Treasury']
['ProShares UltraPro Short 20+ Year Treasury', 'SPDR Gold Shares']
['ProShares UltraPro Short 20+ Year Treasury', 'SPDR Gold Shares', 'Belden Inc.']
['ProShares UltraPro Short 20+ Year Treasury', 'SPDR Gold Shares', 'Belden Inc.', 'Vanguard Real Estate Index Fund']
['ProShares UltraPro Short 20+ Year Treasury', 'SPDR Gold Shares', 'Belden Inc.', 'Vanguard Real Estate Index Fund', 'iShares MSCI Emerging Markets ETF']
['ProShares UltraPro Short 20+ Year Treasury', 'SPDR Gold Shares', 'Belden Inc.', 'Vanguard Real Estate Index Fund', 'iShares MSCI Emerging Markets ETF', 'SPDR S&P 500 ETF Trust']


,자산군,세팅비중,티커,예상투자금,종목명
0,채권,10,TTT,10000000.0,ProShares UltraPro Short 20+ Year Treasury
1,금,20,GLD,20000000.0,SPDR Gold Shares
2,원자재,20,BDC,20000000.0,Belden Inc.
3,부동산,10,VNQ,10000000.0,Vanguard Real Estate Index Fund
4,국내주식,20,EEM,20000000.0,iShares MSCI Emerging Markets ETF
5,해외주식,20,SPY,20000000.0,SPDR S&P 500 ETF Trust


In [44]:
from datetime import datetime, timedelta
def set_current_price(df, sdate, edate):
    flag = True
    ohlcv_df = None
    # while flag:
    #     ohlcv_df = data.get_data_yahoo(date)      
        
    #     if len(ohlcv_df) == 0:
    #         if type(date) == str:
    #             date = datetime.strptime(date, '%Y-%m-%d')
    #         date_obj = date + timedelta(days=1)
    #         date = str(date_obj)[:10]
    #     else:  
    #         print(ohlcv_df.head())         
    #         flag = False

    price = []
    for i in df['티커']:
    #for i,row in enumerate(df.values):  
        print(i)
        df2 = data.get_data_yahoo(i , start = sdate, end = edate)     
        print(df2.head())   
        price.append(df2['Close'][0])       
           
    df['현재가'] = price
    df['투자날짜'] = str(edate)[:10]
    return df

In [ ]:
df = set_current_price(df, '2023-04-10','2023-04-11')
df

In [47]:
import numpy as np

In [49]:
def get_volume(df,cur_kor):    
    df['수량']=df['예상투자금']/(df['현재가']*cur_kor)
    df['수량'].replace(np.inf, 0, inplace=True)
    df['수량']=df['수량'].astype(int)
    return df


In [51]:
df= get_volume(df,1310)
df

,자산군,세팅비중,티커,예상투자금,종목명,현재가,투자날짜,수량
0,채권,10,TTT,10000000.0,ProShares UltraPro Short 20+ Year Treasury,59.630001,2023-04-11,128
1,금,20,GLD,20000000.0,SPDR Gold Shares,185.110001,2023-04-11,82
2,원자재,20,BDC,20000000.0,Belden Inc.,80.529999,2023-04-11,189
3,부동산,10,VNQ,10000000.0,Vanguard Real Estate Index Fund,82.769997,2023-04-11,92
4,국내주식,20,EEM,20000000.0,iShares MSCI Emerging Markets ETF,39.430000,2023-04-11,387
5,해외주식,20,SPY,20000000.0,SPDR S&P 500 ETF Trust,409.609985,2023-04-11,37


In [54]:
def get_monthly_investment(df,cur_kor):
    df['실투자금']=df['현재가']*df['수량']*cur_kor
    return df

In [55]:
df = get_monthly_investment(df, 1310)
df

,자산군,세팅비중,티커,예상투자금,종목명,현재가,투자날짜,수량,실투자금
0,채권,10,TTT,10000000.0,ProShares UltraPro Short 20+ Year Treasury,59.630001,2023-04-11,128,9.998759e+06
1,금,20,GLD,20000000.0,SPDR Gold Shares,185.110001,2023-04-11,82,1.988452e+07
2,원자재,20,BDC,20000000.0,Belden Inc.,80.529999,2023-04-11,189,1.993842e+07
3,부동산,10,VNQ,10000000.0,Vanguard Real Estate Index Fund,82.769997,2023-04-11,92,9.975440e+06
4,국내주식,20,EEM,20000000.0,iShares MSCI Emerging Markets ETF,39.430000,2023-04-11,387,1.998983e+07
5,해외주식,20,SPY,20000000.0,SPDR S&P 500 ETF Trust,409.609985,2023-04-11,37,1.985380e+07


In [56]:
sum(df['실투자금'])

99640760.4825592